In [1]:
import os
from BiasedErasure.delayed_erasure_decoders.Experimental_Loss_Decoder import *
import time
import numpy as np
from statsmodels.stats.proportion import proportion_confint


distance = 5
output_dir = '/Users/gefenbaranes/Lukin SiV Dropbox/Gefen Baranes/local_Repetitive_QEC/Results/CX_experiment'

decoder_basis = 'XX'
gate_ordering = ['N', 'Z']
num_ancilla_blocks = 4
num_rounds = num_ancilla_blocks + 1
measurement_events = np.load(f'/Users/gefenbaranes/Lukin SiV Dropbox/Gefen Baranes/local_Repetitive_QEC/Oct24/measurement_events_X_NZZrNr_2024_10_06.npy')
num_of_shots = len(measurement_events)

train_measurement_events = measurement_events[::2]
test_measurement_events = measurement_events[1::2]

measurement_events = measurement_events[:100]
# measurement_events = test_measurement_events

print(f"len(measurement_events) = {len(measurement_events)}")

noise_params = {}

Meta_params = {'architecture': 'CBQC', 'code': 'Rotated_Surface', 'logical_basis': decoder_basis,
            'bias_preserving_gates': 'False',
            'noise': 'atom_array', 'is_erasure_biased': 'False', 'LD_freq': '1000', 'LD_method': 'None',
            'SSR': 'True', 'cycles': str(num_rounds - 1),
            'ordering': gate_ordering,
            'decoder': 'MLE',
            'circuit_type': 'logical_CX_2', 'Steane_type': 'None', 'printing': 'True', 'num_logicals': '2',
            'loss_decoder': 'independent',
            'obs_pos': 'd-1', 'n_r': '0'}

print(gate_ordering)

use_loss_decoding = True  # if False: use same DEM every shot, without utilizing SSR.
use_independent_decoder = True  # if False: in every lifecycle, we just apply supercheck at the end. If True: we count the full lifecycle with different potential loss locations and corresponding Clifford propagations.
use_independent_and_first_comb_decoder = False  # This is relevant only if use_independent_decoder=True. If False: use only independent lifecycles. If True: adds a single combination of lifecycles to the decoder.


simulate_data = True

num_shots = 2 # Change #1: if simulate_data = True, you can set num_shots, and put as input measurement_events = None
# predictions, observable_flips, dems_list = Loss_MLE_Decoder_Experiment(Meta_params, distance, distance, output_dir,
#                                                                     measurement_events,
#                                                                     None, use_loss_decoding,
#                                                                     use_independent_decoder,
#                                                                     use_independent_and_first_comb_decoder,
#                                                                     simulate_data=simulate_data, logical_gaps=False, 
#                                                                     noise_params=noise_params, num_shots=num_shots)





# logical_probability = np.mean(np.logical_xor(observable_flips, predictions))

# logical_probability = np.mean(np.logical_xor(predictions[:, 0], observable_flips.flatten())).squeeze()
# print(r'Probability of logical +1: %.6f' % (1 - logical_probability))

len(measurement_events) = 100
['N', 'Z']


In [4]:
num_shots = 10
# Change #2: a function that doesn't decode, just gives you the measurements, detection events, observables and the circuit:
measurement_events_all_shots, detection_events_all_shots, observable_flips_all_shots, LogicalCircuit = get_simulated_measurement_events(Meta_params, distance, distance, num_shots, noise_params)

num_CX_per_layer = 2
Using orderings: ['N', 'Z']
sampling for the following loss pattern: [   4  110  210 2849 3591 3805 3823 3841]
{2: [6], 18: [22], 25: [22], 253: [51], 316: [83], 329: [5, 43], 347: [75]}
dem = error(0.006241303902690729585) D0
error(0.0007214161525332775729) D0 D3
error(0.0003842964596437939005) D0 D3 D50
error(0.003262451737979945362) D0 D4
error(0.002870930643198536369) D0 D4 D14 D16
error(0.000365692573460413956) D0 D4 D14 D16 D50
error(0.002093106362788960725) D0 D4 D27
error(1.860542025217774899e-05) D0 D4 D50
error(0.002870930643198536369) D0 D12
error(0.000365692573460413956) D0 D12 D50
error(0.00347959372773112537) D0 D24
error(0.001738259664894761022) D0 D24 D26
error(0.00161747827147220094) D0 D24 D26 D27
error(0.0004858736298733305238) D0 D24 D27
error(0.07090495828555924696) D0 D26
error(0.001254660000000000107) D0 D26 D27
error(0.00174021585300375382) D0 D27
error(0.0004858736298733305238) D0 D27 D33
error(6.306094021746620282e-05) D0 D33
error(0.00040

In [8]:
detection_events_all_shots.shape

(10, 96)

In [9]:
detection_events_all_shots

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0,